In [1]:
!pip3 install pandas
!pip3 install --upgrade tensorflow-gpu

import numpy as np
import pandas as pd
import sys
import glob
import os

import tensorflow as tf
from tensorflow.keras.losses import mse
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Lambda, Input, Dense
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers

from sklearn.metrics import accuracy_score

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


2022-10-03 21:17:05.001291: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-03 21:17:05.100254: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-03 21:17:05.100268: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-03 21:17:05.120670: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-03 21:17:05.541999: W tensorflow/stream_executor/pla

In [2]:
def get_files(path):
    all_files = glob.glob(os.path.join(path , '*.csv'))

    files_list = []
    for file in all_files:
        df = pd.read_csv(file, index_col = None, encoding = 'utf-8', sep = ',', low_memory = False)
        files_list.append(df)
        
    df = pd.concat(files_list, axis = 0, ignore_index = True)
    return df

In [3]:
# Reparameterization trick

def reparametrization(args):
    z_mean, z_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape = (batch, dim))
    
    return z_mean + K.exp(0.5 * z_var) * epsilon

# Sample the normally distributed z - mean + sigma * epsilon. The epsilon ensures the continuity of latent space and helps
# the network to keep correcting its parameters through backpropagation

In [4]:
# Get error term

def get_error_term(v1, v2, _rmse = True):
    if _rmse:
        return np.sqrt(np.mean((v1 - v2) ** 2, axis = 1))
    
    return np.mean(abs(v1 - v2), axis = 1)

# Calculates the error between the original vector and the predicted one

In [5]:
# Encoder Model

def vae_encoder(input_shape, intermediate_dim, latent_dim, reparametrization):
    inputs = Input(shape = input_shape, name = 'encoder_input')
    x = Dense(intermediate_dim, activation = 'relu')(inputs)

    z_mean = Dense(latent_dim, name = 'z_mean')(x)
    z_var = Dense(latent_dim, name = 'z_var')(x)
    z = Lambda(reparametrization, output_shape = (latent_dim,), name = 'z')([z_mean, z_var])

    encoder = Model(inputs, z, name = 'encoder')
    return inputs, encoder, z_var, z_mean

# The encoder learns a function that takes an input array of size n and can generate two vectors that represents the
# parameters (mean and variance) of a distribution from which the latent vector is sampled.

# encoder(input_vector[]) => latent_v_mu[], latent_v_lvar[]
# So that - latent_v[0] ~  N(latent_v_mu[0], latent_v_lvar[0])
# and latent_v[1] ~  N(latent_v_mu[1], latent_v_lvar[1])

In [6]:
# Decoder model

def vae_decoder(intermediate_dim, latent_dim, original_dim):
    latent_inputs = Input(shape = (latent_dim,), name = 'z_sampling')
    x = Dense(intermediate_dim, activation = 'relu')(latent_inputs)
    outputs = Dense(original_dim, activation = 'sigmoid')(x)

    # Instantiate the decoder model

    decoder = Model(latent_inputs, outputs, name = 'decoder')
    return decoder

# Transforms the latent feature space composed by distributions of mean and variance back to the original input vector

In [7]:
def maximize_training(X_train, X_test, Y_test, inputs, outputs, vae_loss, dict_params, anomaly_threshold):    
    # Create Model
    opt = optimizers.Nadam(learning_rate = dict_params['learning_rate'], beta_1 = 0.9, beta_2 = 0.999, epsilon = 1e-08, schedule_decay = 0.004)  
    vae_model = Model(inputs, outputs, name = 'vae_mlp')
    vae_model.compile(optimizer = opt, loss = vae_loss)
    
    # Train
    history = vae_model.fit(X_train, X_train,
                            shuffle = True,
                            epochs = int(dict_params['epochs']),
                            batch_size = int(dict_params['batch_size'])
    
    # Maximize accuracy value
    X_pred = vae_model.predict(X_test)
    error_vector = get_error_term(X_pred, X_test, _rmse=False)
    attacks = (error_vector > anomaly_threshold) 
    accuracy = accuracy_score(Y_test, attacks)
    
    return accuracy